<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/LLM_LLaMA_SafeLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12=

LLaMA

In [ ]:
!pip -q install sentencepiece
!pip -q install transformers peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
# Replace 'your_huggingface_token' with your actual token
HUGGINGFACE_TOKEN = "your token"

# Authenticate with Hugging Face
login(HUGGINGFACE_TOKEN)

print("✅ Successfully logged into Hugging Face!")

✅ Successfully logged into Hugging Face!


With quantization LoRA

In [ ]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer, AdamW
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

# Set random seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LLaMA 2.7B tokenizer and model
model_name = "Meta-Llama/Llama-2-7b-chat-hf"
# tokenizer = LlamaTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Load the correct tokenizer for LLaMA-2 using AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Set padding token to EOS if missing
tokenizer.pad_token = tokenizer.eos_token

# base_model = LlamaForCausalLM.from_pretrained(
#     model_name, torch_dtype=torch.float16, device_map="auto"
# )

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized Float 4 (better than standard FP4)
    bnb_4bit_use_double_quant=True,  # Uses secondary quantization for better precision
    bnb_4bit_compute_dtype=torch.float16  # Keeps computation in FP16 for stability
)

# uncomment these first time
# Load LLaMA 2.7B with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

base_model.config.pad_token_id = tokenizer.eos_token_id  # Set pad token ID

# Configure LoRA for memory-efficient fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Apply LoRA to attention layers
)

# Wrap the model with LoRA adapters
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # Verify LoRA trainable parameters

# Define QA training and validation samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

# Preprocessing function
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)

    # Copy input_ids to labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore these tokens in loss computation

    inputs["labels"] = labels
    return inputs

# Convert datasets and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# DataLoader Collation
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

# Create DataLoaders
batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-lora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels for loss computation
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute Loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)


import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best"

# Load the base LLaMA 2.7B model
# base_model = LlamaForCausalLM.from_pretrained(
#     "Meta-Llama/Llama-2-7b-chat-hf",
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Llama-2-7b-chat-hf",
#     quantization_config=bnb_config,
#     device_map="auto"
# )

# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned tokenizer
# tokenizer = LlamaTokenizer.from_pretrained(best_model_path)
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20, Train Loss: 1.3395, Val Loss: 1.1618
✅ Best model saved at epoch 1 with validation loss: 1.1618
Epoch 2/20, Train Loss: 0.9050, Val Loss: 0.9045
✅ Best model saved at epoch 2 with validation loss: 0.9045
Epoch 3/20, Train Loss: 0.9064, Val Loss: 0.6462
✅ Best model saved at epoch 3 with validation loss: 0.6462
Epoch 4/20, Train Loss: 0.5318, Val Loss: 0.4623
✅ Best model saved at epoch 4 with validation loss: 0.4623
Epoch 5/20, Train Loss: 0.2958, Val Loss: 0.3614
✅ Best model saved at epoch 5 with validation loss: 0.3614
Epoch 6/20, Train Loss: 0.2159, Val Loss: 0.2995
✅ Best model saved at epoch 6 with validation loss: 0.2995
Epoch 7/20, Train Loss: 0.2165, Val Loss: 0.2550
✅ Best model saved at epoch 7 with validation loss: 0.2550
Epoch 8/20, Train Loss: 0.1603, Val Loss: 0.2218
✅ Best model saved at epoch 8 with validation loss: 0.2218
Epoch 9/20, Train Loss: 0.1552, Val Loss: 0.1919
✅ Best model saved at epoch 9 with validation loss: 0.1919
Epoch 10/20, Train Loss: 0.1

In [ ]:
# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only short answer of the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

Q: Provide only short answer of the question: Who created the Mona Lisa?
A: Leonardo da Vinci.


In [ ]:
!git clone https://github.com/IBM/SafeLoRA.git
%cd SafeLoRA

Cloning into 'SafeLoRA'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
Receiving objects: 100% (12/12), 9.36 KiB | 9.36 MiB/s, done.
Resolving deltas: 100% (4/4), done.
remote: Total 12 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
/content/SafeLoRA


In [ ]:
%cd SafeLoRA
!ls

/content/SafeLoRA
config.py  __init__.py	LICENSE  model.py  __pycache__	README.md


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from config import SafeLoRAConfig
from model import SafeLoRA

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized Float 4 (better than standard FP4)
    bnb_4bit_use_double_quant=True,  # Uses secondary quantization for better precision
    bnb_4bit_compute_dtype=torch.float16  # Keeps computation in FP16 for stability
)

model_name = "Meta-Llama/Llama-2-7b-chat-hf"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True
)

best_model_path = "/content/llama-lora-qa-best"
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

SafeLoRAConfig.base_model_path = model_name  #you should modify the path
SafeLoRAConfig.aligned_model_path = model_name #you should modify the path
safelora = SafeLoRA(model, SafeLoRAConfig)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]